In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, make_scorer, confusion_matrix, precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
import lightgbm as lgb

In [19]:
#lettura csv
filepath = r"C:\Users\lucap\OneDrive\Desktop\PhD\Racheal\Dati\Walden 97-19 integrated v2.csv"
df = pd.read_csv(filepath, sep = ";" )
#df = df[df['m_anno_indagine'] > 2016] #seleziono le righe che riguardano l'osservatorio dal 2019 in poi
df = df[df['m_anno_indagine']>=2017].drop_duplicates(subset='IDU', keep="last")
df = df[
   df.columns[ df.isna().sum() <= 0.3 * len(df) ] ]#seleziono solo le colonne hanno valori Nan inferiori al 30%
# Rimuovere gli spazi bianchi da tutte le colonne
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

le = LabelEncoder()
df['m_sesso'] = le.fit_transform(df['m_sesso'])
df['m_p_pubblico_privato'] = le.fit_transform(df['m_p_pubblico_privato'])
df['m_p_nascita_in_italia_genitori'] = le.fit_transform(df['m_p_nascita_in_italia_genitori'])

C:\Users\lucap\AppData\Local\Temp\ipykernel_2880\208190578.py:3: DtypeWarning: Columns (0,4,8,9,10,11,12,13,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,44,45,46,47,49,50,51,52,53,54,55,61,62,63,64,65,66,68,69,70,71,78,80,82,84,85,86,87,91,93,94,96,97,98,99,101,102,103,105,106,108,109,110,111,113,114,115,117,118,119,120,121,123,124,125,126,128,129,130,132,133,134,136,137,138,139,140,142,143,144,145,146,147,148,150,151,153,154,155,157,158,159,162,164,165,166,167,169,170,171,172,173,174,175,176,177,178,179,180,181,182,184,185,186,187,188,189,190,191,192,194,195,196,197,198,199,200,201,202,203,204,205,207,209,211,212,213,214,215,217,218,219,221,222,223,224,225,226,227,228,229,230,231,232,233,235,236,239,240,242,243,245,246,248,249,250,251,252,253,254,255,256,257,258,261,263,264,265,266,267,272,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,292,293,294,295,296,297,298,299,300,301,302,303,305,306,307,309,310,311,312,313,314,316,317,318,320,321,32

In [20]:
varsToDrop = ['IDU','m_pesos','m_peso',"m_p_cittadinanza", "m_p_data_nascita", "m_modalita_telefono_vs_online",
    'm_p_comune_istat',
                'm_p_data_nascita','m_p_eta_6',
                    'm_p_in_ita_dal','m_p_origini',
                            'm_sr_professione','m_sr_statusSocioEconomico_ceto',"m_p_autocol"
                            ] # rimuovo regioni perchè uso one hot e non voglio aumentare troppo il numero di colonne
df= df.drop(varsToDrop, axis=1)


In [21]:
varsToDrop_val = ["m_op_unita_nazionale", 'm_op_costituzione.1','m_op_miglioramento_rapp_stato_cittadini','m_op_ventennio',
'm_op_criminalita_ultimiAnni_doveVive','m_op_cambio_legge_aborto','m_op_organizzazioneEuropa_futuro', 'm_op_fiducia_in_sindacati','m_p_iscrizione_sindacato'
,'m_op_tensioneSociale_italianiVSextracom_propriaCitta','m_op_sistemaScolasticoItaliano','m_op_reazione_a_discussioni_politiche',
'm_op_fiducia_negli_altri','m_op_inclusione_societa','m_op_margineIntervento_vita','m_op_paure_della_vecchiaia','m_fiducia_proprieIdee_rispetto_avvenimentiMondo'
,'m_op_origine_maggiorImpulso_ripresa_italia', 'm_op_profilo_socioCulturale_vs_genitori','m_op_causePrincipali_violenzaDonne_1','m_op_causePrincipali_violenzaDonne_2',
'm_op_circostanze_cheGiustificano_violenzaMoglie','m_op_priorita_per_affrontare_violenzaDonne_1','m_op_priorita_per_affrontare_violenzaDonne_2', 'm_TREND_15_FASCISMO NEW',
'm_op_quanto_populista_PD','m_op_quanto_populista_M5S','m_op_quanto_populista_Lega','m_op_quanto_populista_FI','m_op_definizione_populismo',
'm_op_caratteristiche_dei_politici_apprezzati','m_op_cosa_contradistingue_comunita','m_op_pericoliEdisagi_figli_cheSpaventano_genitori_04',
'm_op_pericoliEdisagi_figli_cheSpaventano_genitori_03','m_op_pericoliEdisagi_figli_cheSpaventano_genitori_02','m_op_pericoliEdisagi_figli_cheSpaventano_genitori_01',
'm_op_definizione_bene_comune','m_op_comunita_appartenenza_01','m_p_zona_5istat','m_op_rimanere_in_propriaZona_vs_spostarsi',
'm_ac_preparazione_nuoveGenerazioni_piuScadente','m_op_attualita_concetti_sinistraCentroDestra','m_op_importanza_essereItaliano',
'm_op_attitudine_italiani_fascismo']
df= df.drop(varsToDrop_val, axis=1)

In [23]:
df.shape

(4504, 161)

In [24]:
save_path = r"C:\Users\lucap\OneDrive\Desktop\PhD\Racheal\Dati\dataset_clean.csv" 

In [25]:
df.to_csv(
    save_path,
    index=False,        # niente indice nel CSV
    encoding="utf-8",   # evita problemi di caratteri accentati
    sep=";"             # se preferisci il separatore semicolon
)